In [ ]:
import pandas as pdfrom sqlalchemy import
(create_engine,inspect,text,select,MetaData,Table,and_,or_,desc,asc,func,case,cast,Float,)

In [ ]:
# dialtect+driver:///username:password@host:port/database.db - connecting to an outside db
 
# dialect+driver:////filepath.db
 
chinook_engine = create_engine("sqlite+pysqlite://///workspaces/ERN-sessions/SQL Learning/chinook.db")
 
inspect(chinook_engine).get_table_names()

In [ ]:
# gravity_engine
gravity_engine = create_engine("sqlite+pysqlite:////workspaces/ERN-sessions/SQL Learning/gravity.db")
inspect(gravity_engine).get_table_names()

In [ ]:
chinook_metadata = MetaData()
 
tracks = Table('tracks', chinook_metadata, autoload_with=chinook_engine)
albums = Table('albums', chinook_metadata, autoload_with=chinook_engine)
artists = Table('artists', chinook_metadata, autoload_with=chinook_engine)

In [ ]:
chinook_metadata = MetaData()
 
tracks = Table('tracks', chinook_metadata, autoload_with=chinook_engine)
albums = Table('albums', chinook_metadata, autoload_with=chinook_engine)
artists = Table('artists', chinook_metadata, autoload_with=chinook_engine)
 
# do this for the gravity.db we want: book, author, book_author
# make a metadata obj. for gravity, make the three tables

In [ ]:
# Question: if the table is within a database in an sql server, can one use sqllite?
 
# Answer (Will): I think you could, but I've only used postgresql, you'd have to have a tinker I think.
# sqlite is really for spinning up a db in local memory rather than making actual server connections

In [ ]:
gravity_metadata = MetaData()
 
books = Table('book', gravity_metadata, autoload_with=gravity_engine)
authors = Table('author', gravity_metadata, autoload_with=gravity_engine)
book_authors = Table('book_author', gravity_metadata, autoload_with=gravity_engine)

In [ ]:
with chinook_engine.connect() as con:
    stmt = select(tracks.c.Name, (tracks.c.Bytes / tracks.c.Milliseconds).label('Bytes per Millisecond'))
    result_proxy = con.execute(stmt)
    result = result_proxy.fetchmany(size=10)
 
result_df = pd.DataFrame(result)
result_df

In [ ]:
# case(when this is true, do this)
# if we wrap case in function, we can do the function only when the case is true
 
with chinook_engine.connect() as con:
    album_1_length = func.sum(case((tracks.c.AlbumId == 1, tracks.c.Milliseconds), else_=0))
    stmt = select(album_1_length / ( 1000 * 60 ) )
    result_proxy = con.execute(stmt)
    result = result_proxy.scalar()
 
print(result)
 
 
# work out length of album 1 as a percentage of entire library
# clause to get length of album 1
# clause to get length of entire library
# stmt to select the calcualtion to get length of album 1 over entire libraryr ad pct

In [ ]:
# work out length of album 1 as a percentage of entire library
# clause to get length of album 1
# clause to get length of entire library
# stmt to select the calcualtion to get length of album 1 over entire libraryr ad pct
 
with chinook_engine.connect() as con:
    album_1 = func.sum(case((tracks.c.AlbumId == 1, tracks.c.Milliseconds), else_=0))
    entire_lib = func.sum(tracks.c.Milliseconds)
    stmt = select((album_1 / entire_lib) * 100)
    result_proxy = con.execute(stmt)
    result = result_proxy.scalar()
 
print(result)

In [ ]:
# if there's a pre-determined relationship
with chinook_engine.connect() as con:
    stmt = select(artists.c.Name, tracks)
    result_proxy = con.execute(stmt)
    result = result_proxy.fetchmany(size=10)
 
# result_df = pd.DataFrame(result)
# result_df
 
# no pre-determined
with chinook_engine.connect() as con:
    stmt = select(tracks.join(albums, tracks.c.AlbumId == albums.c.AlbumId))
    result_proxy = con.execute(stmt)
    result = result_proxy.fetchmany(size=20)
 
result_df = pd.DataFrame(result)
result_df

In [ ]:
# 1. select a new table of book title with author name - using pre-detemrined
# 2. make a big table joining books to book_authors to authors - determine yourself

In [ ]:
with gravity_engine.connect() as con:
    stmt = select(books.join(book_authors, books.c.book_id == book_authors.c.book_id).join(authors))
    result_proxy = con.execute(stmt)
    result = result_proxy.fetchmany(size=10)
 
result_df = pd.DataFrame(result)
result_df

In [ ]:
# The task is to use SQLAlchemy and pandas and, any way you want,
# find out: The name of the author who has the most books in the database, and
# what those books are
# # A stretch task for homework is to find out how many orders have been
# placed which contain books by that author
# for the stretch task, email me your answers.